### 데이터 수집
- 데이터셋 1 : 재무비율 데이터
    - KOSPI 200 안에 2015~2024년까지 한번이라도 들어온 기업에 대해서 진행
- 데이터셋 2 : 주가 데이터
    - 시가 데이터와 종가 데이터를 활용해서 진행

## 1. 라이브러리 호출

In [33]:
import pandas as pd
from pykrx import stock
from datetime import datetime, timedelta
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## 2. 데이터 수집
### 2.1 주가 데이터 (PyKrx)

##### 지정한 날짜가 영업일이 아닐 때 다음 영업일을 찾아 주는 함수

In [34]:
def get_previous_trading_day(date):
    """
    입력 날짜가 휴장일이면 다음 영업일까지 반복해서 이동
    date: 'YYYYMMDD' 문자열
    """
    while True:
        # 삼성전자(005930)의 OHLCV 데이터 확인
        df = stock.get_market_ohlcv_by_date(date, date, "005930")
        if not df.empty:
            return date
        # 휴장일이면 이전날로 이동
        dt = datetime.strptime(date, "%Y%m%d") - timedelta(days=+1)
        print(f"{date}는 휴장일입니다. 이전 영업일을 검색합니다...")
        date = dt.strftime("%Y%m%d")

##### 지정한 기간동안의 코스피200의 종목 리스트를 반환하는 함수

In [35]:
def kospi200_list(start_year, end_year):
    # 결과를 저장할 리스트 생성
    records = []

    for year in range(start_year-1, end_year + 1):
        date = f"{year}0430"  # 4월말 기준 (휴장일일 경우 이후 영업입)
        trading_date = get_previous_trading_day(date)
        df = stock.get_index_portfolio_deposit_file("1028", trading_date, alternative = True)
        # 각 코드에 대해 연도와 함께 리스트에 저장
        for code in df:
            records.append({'연도': year, '주식코드': code})

    # 리스트를 데이터프레임으로 변환
    result_df = pd.DataFrame(records)

    frequency = result_df['주식코드'].value_counts()
    frequency_df = frequency.reset_index()
    frequency_df.columns = ['주식코드', '빈도수']
    code_df = frequency_df[['주식코드']]
    code_list = code_df['주식코드'].tolist()

    return code_list

##### 기간 내 코스피 200 종목 리스트 수집

In [36]:
# 코스피200 종목 리스트 수집 
ticker_list = kospi200_list(2015, 2023)

# 누락된 2014년 코스피200 종목 데이터 수집 후 병합
for year in range(2008, 2015):
    list_df = pd.read_excel(f"data/index_data/{year}_04_kospi200.xlsx",dtype={"종목코드" : str})
    list_df["날짜"] = f"{year}-04-30"
    list_df["연도"] = f"{year}"
    list_df = list_df[["종목코드","날짜","연도","상장시가총액"]]
    list_df.rename(columns={'상장시가총액' : '시가총액', '종목코드' : '주식코드'}, inplace=True)
    code_list = list_df["주식코드"].to_list()
    ticker_list = ticker_list+code_list
ticker_list = list(set(ticker_list))

KRX web server does NOT provide data prior to 2014/05/01.
20160430는 휴장일입니다. 이전 영업일을 검색합니다...
20170430는 휴장일입니다. 이전 영업일을 검색합니다...
20170429는 휴장일입니다. 이전 영업일을 검색합니다...
20200430는 휴장일입니다. 이전 영업일을 검색합니다...
20220430는 휴장일입니다. 이전 영업일을 검색합니다...
20230430는 휴장일입니다. 이전 영업일을 검색합니다...
20230429는 휴장일입니다. 이전 영업일을 검색합니다...


##### 수익률 컬럼 생성

In [38]:
start_year = 2013
end_year = 2025

records = []
for year in range(start_year, end_year+1):
    start_date = f"{year}0430"
    trading_date = get_previous_trading_day(start_date)
    for code in ticker_list:
        cap_df = stock.get_market_ohlcv(trading_date, trading_date, code, adjusted=True)
        if trading_date in cap_df.index:
            close = cap_df.loc[trading_date, '종가']
        # open = cap_df.loc[code, '시가']
            records.append({
                '주식코드': code,
                '연도': year,
                '날짜': trading_date,
                '종가': close,
                # '시가': open
            })
        else:
            records.append({
                '주식코드': code,
                '연도': year,
                '날짜': trading_date,
                '종가': None,
                # '시가': None
            })

20160430는 휴장일입니다. 이전 영업일을 검색합니다...
20170430는 휴장일입니다. 이전 영업일을 검색합니다...
20170429는 휴장일입니다. 이전 영업일을 검색합니다...
20200430는 휴장일입니다. 이전 영업일을 검색합니다...
20220430는 휴장일입니다. 이전 영업일을 검색합니다...
20230430는 휴장일입니다. 이전 영업일을 검색합니다...
20230429는 휴장일입니다. 이전 영업일을 검색합니다...


In [40]:
for year in range(2008, 2013):
    df = pd.read_excel(f"data/close_data/{year}04_close.xlsx", dtype={"종목코드" : str})
    filtered_df = df[df['종목코드'].isin(ticker_list)]
    for _, row in filtered_df.iterrows():
        records.append({
            '주식코드': row['종목코드'],
            '연도': year,
            '날짜': f"{year}0430",
            '종가': np.float64(row['종가'])
        })

In [66]:
copy_records = records.copy()

##### 수익률컬럼 생성 후 -1, -inf, +inf 결측값으로 처리 후 제거

In [86]:
df = pd.DataFrame(records)
df.sort_values(by=['주식코드','연도'],ascending=True,ignore_index=True,inplace=True)
df = df.drop_duplicates()

# 공시년월 이후 1년간 수익률 컬럼 생성 
# ex) 2014년 재무는 2015년 4월에 나옴 고로, 수익률은 2015.04~2016.04 의 수익률을 계산하여 연도가 2014인 row에 포함됨
df['수익률'] = df.groupby('주식코드')['종가'].pct_change(1).shift(-2)

df = df.replace([np.inf, -np.inf], np.nan)

df = df[df['수익률'] != -1]

df = df[['주식코드','연도','수익률']]
df.dropna(inplace=True)

##### PyKrx 수익률 xlsx파일 저장

In [88]:
df.to_excel("data/krx.xlsx", index=False)

### 2.2 재무재표 데이터 (ts2000)

##### ts2000 데이터 불러오기

In [89]:
tsdata_df = pd.read_excel('data/ts2000.xlsx', dtype={'거래소코드' : str})

##### 결측치 제거 - 회계기준이 다른 금융업 제외 후 제조업만 사용

In [90]:
tsdata_df.dropna(inplace=True)

##### 조인을 위한 ts2000 데이터 전처리

In [91]:
def ts_data_loader(df):
    ts_df = df.copy()
    ts_df['회계년도'] = ts_df['회계년도'].str.split('/').str[0].astype(int)
    ts_df['거래소코드'] = ts_df['거래소코드'].str.strip()
    ts_df.rename(columns = {'회계년도':'연도',
                            '거래소코드':'주식코드'}, inplace=True)

    # 중복 확인
    duplicates = ts_df.duplicated(subset=['주식코드', '연도'], keep=False)
    ts_df[duplicates]

    # 중복된 ('주식코드', '연도') 조합에서 첫 번째 값만 남김
    ts_df = ts_df.drop_duplicates(subset=['주식코드', '연도'], keep='first').sort_values(by=['주식코드','연도'])

    return ts_df

ts_df = ts_data_loader(tsdata_df)

##### 데이터 eda결과 drop할 컬럼들 목록

In [92]:
drop_cols = ['노동장비율(IFRS)','기계장비율(IFRS)','자본집약도(IFRS)',
'비유동생물자산증가율(IFRS)','R & D 투자효율(IFRS)','세금과공과 대 총비용비율(IFRS)',
'사내유보율(IFRS)','이자보상배율(이자비용)(IFRS)','이자보상배율(순금융비용)(IFRS)','비유동자산집중도(IFRS)']

ts_df.drop(columns = drop_cols, inplace = True)

##### (원)단위 컬럼 제거 - 비율 척도 계산하기 위해

In [93]:
col_list = []
for col in ts_df.columns:
    if '원)' in col:
        col_list.append(col)
ts_df.drop(columns=col_list,inplace=True)

##### 결산월이 12월이 아닌 데이터는 제거 - 데이터 통일성을 위해

In [94]:
flag = ts_df['결산월'] != 12
ts_df = ts_df.loc[~flag]

## 3. PyKrx 데이터 ts2000 데이터 조인

##### Pykrx 데이터와 ts2000 데이터 합치기

In [95]:
# krx.xlsx 데이터 불러오기
krx_df = pd.read_excel("data/krx.xlsx", dtype={'주식코드' : str})

In [96]:
ts_rtn_df = pd.merge(ts_df, krx_df, on=['주식코드','연도'], how='inner')

##### 결산월 컬럼은 factor_analysis_with_rlm 과정과 backtesting 과정에 필요없는 컬럼이기 때문에 삭제

In [97]:
ts_rtn_df = ts_rtn_df.drop(columns='결산월')

##### 분석 위한 최종 데이터 저장

In [98]:
ts_rtn_df.to_excel("data/ts_rtn.xlsx", index=False)